In [1]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = [8, 3]
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels

import scipy
from scipy.stats import pearsonr

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
print(matplotlib.__version__)
print(pd.__version__)
print(np.__version__)
print(statsmodels.__version__)
print(scipy.__version__)


## Obtain and visualize data

In [ ]:
## data obtained from https://datahub.io/core/global-temp#data
df = pd.read_csv("global_temps.csv")
df.head()

In [ ]:
df.Mean[:100].plot()

## Exercise: what is wrong with the data and plot above? How can we fix this?

In [ ]:
 df = df.pivot(index='Date', columns='Source', values='Mean')

In [ ]:
df.head()

In [ ]:
df.GCAG.plot()

In [ ]:
type(df.index)

## Exercise: how can we make the index more time aware?

In [ ]:
df.index = pd.to_datetime(df.index)

In [ ]:
type(df.index)

In [ ]:
df.GCAG.plot()

In [ ]:
df['1880']

In [ ]:
plt.plot(df['1880':'1950'][['GCAG', 'GISTEMP']])

In [ ]:
plt.plot(df['1950':][['GISTEMP']])

## Exercise: How strongly do these measurements correlate contemporaneously? What about with a time lag?

In [ ]:
plt.scatter(df['1880':'1900'][['GCAG']], df['1880':'1900'][['GISTEMP']])

In [ ]:
plt.scatter(df['1880':'1899'][['GCAG']], df['1881':'1900'][['GISTEMP']])

In [ ]:
pearsonr(df['1880':'1899'].GCAG, df['1881':'1900'].GISTEMP)

In [ ]:
df['1880':'1899'][['GCAG']].head()

In [ ]:
df['1881':'1900'][['GISTEMP']].head()

In [ ]:
min(df.index)

In [ ]:
max(df.index)

## Unobserved component model

In [ ]:
train = df['1960':]

### model parameters

In [ ]:
# smooth trend model without seasonal or cyclical components
model = {
    'level': 'smooth trend', 'cycle': False, 'seasonal': None, 
}


### fitting a model

In [ ]:
# https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.structural.UnobservedComponents.html
gcag_mod = sm.tsa.UnobservedComponents(train['GCAG'], **model)
gcag_res = gcag_mod.fit()

In [ ]:
fig = gcag_res.plot_components(legend_loc='lower right', figsize=(15, 9));

## Plotting predictions

In [ ]:
# Perform rolling prediction and multistep forecast
num_steps = 20
predict_res = gcag_res.get_prediction(dynamic=train['GCAG'].shape[0] - num_steps)

predict = predict_res.predicted_mean
ci = predict_res.conf_int()

In [ ]:
plt.plot(predict)

In [ ]:
plt.scatter(train['GCAG'], predict)

In [ ]:
fig, ax = plt.subplots()
# Plot the results
ax.plot(train['GCAG'], 'k.', label='Observations');
ax.plot(train.index[:-num_steps], predict[:-num_steps], label='One-step-ahead Prediction');

ax.plot(train.index[-num_steps:], predict[-num_steps:], 'r', label='Multistep Prediction');
ax.plot(train.index[-num_steps:], ci.iloc[-num_steps:], 'k--');

# Cleanup the image
legend = ax.legend(loc='upper left');

In [ ]:
fig, ax = plt.subplots()
# Plot the results
ax.plot(train.index[-40:], train['GCAG'][-40:], 'k.', label='Observations');
ax.plot(train.index[-40:-num_steps], predict[-40:-num_steps], label='One-step-ahead Prediction');

ax.plot(train.index[-num_steps:], predict[-num_steps:], 'r', label='Multistep Prediction');
ax.plot(train.index[-num_steps:], ci.iloc[-num_steps:], 'k--');

# Cleanup the image
legend = ax.legend(loc='upper left');

## Exercise: consider adding a seasonal term for 12 periods for the model fit above. Does this improve the fit of the model?

In [ ]:
seasonal_model = {
    'level': 'local linear trend',
    'seasonal': 12
}
mod = sm.tsa.UnobservedComponents(train['GCAG'], **seasonal_model)
res = mod.fit(method='powell', disp=False)

In [ ]:
fig = res.plot_components(legend_loc='lower right', figsize=(15, 9));

## How does this compare to the original model?

In [ ]:
pearsonr(gcag_res.predict(), train['GCAG'])

In [ ]:
np.mean(np.abs(gcag_res.predict() - train['GCAG']))

In [ ]:
np.mean(np.abs(res.predict() - train['GCAG']))

## Explore the seasonality more

In [ ]:
seasonal_model = {
    'level': 'local level',
    'seasonal': 12
}
llmod = sm.tsa.UnobservedComponents(train['GCAG'], **seasonal_model)
ll_level_res = llmod.fit(method='powell', disp=False)

In [ ]:
fig = ll_level_res.plot_components(legend_loc='lower right', figsize=(15, 9));

In [ ]:
np.mean(np.abs(ll_level_res.predict() - train['GCAG']))

In [ ]:
train[:48].GCAG.plot()

## Exercise: a common null model for time series is to predict the value at time t-1 for the value at time t. How does such a model compare to the models we fit here?

### Consider correlation

In [ ]:
pearsonr(ll_level_res.predict(), train['GCAG'])

In [ ]:
pearsonr(train['GCAG'].iloc[:-1, ], train['GCAG'].iloc[1:, ])

### What about mean absolute error?

In [ ]:
np.mean(np.abs(ll_level_res.predict() - train['GCAG']))

In [ ]:
np.mean(np.abs(train['GCAG'].iloc[:-1, ].values, train['GCAG'].iloc[1:, ].values))